# Transformer Setup & Training

## 🎯 Concept Primer
Fine-tune TinyBERT or RoBERTa for medical text classification.

**Expected:** Pretrained transformer fine-tuned on medical text

## 📋 Objectives
1. Load pretrained transformer
2. Fine-tune on medical text
3. Evaluate on validation set

## 🔧 Setup

In [10]:
# TODO 1: Import libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report





## 🤖 Load Transformer

### TODO 2: Load model and tokenizer

**Options:** TinyBERT, RoBERTa, DistilBERT

In [11]:
# TODO 2: Load transformer
model_name = 'dmis-lab/biobert-base-cased-v1.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_csv('../data/processed/specialty_taxonomy_v1.csv')

# 🚀 SPEED HACK: Use 20% subset for faster experimentation
print(f"Original dataset: {len(df)} samples")
df = df.groupby('specialty', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)).reset_index(drop=True)
print(f"Sampled dataset: {len(df)} samples (20% of each class)")
print(f"⚡ Expected speedup: ~5x faster training!\n")

unique_specialities = df['specialty'].unique()
label2idx = {label: idx for idx, label in enumerate(unique_specialities)}

df['label_encoded'] = df['specialty'].map(label2idx)

# Use RAW text (not cleaned!)
texts = df['text'].tolist()  # ← Original text!
labels = df['label_encoded'].tolist() 

# Split data (same as baseline)
X_temp, X_test, y_temp, y_test = train_test_split(
    texts, 
    labels,  # You created this in Notebook 03!
    test_size=0.2, 
    random_state=42, 
    stratify=df['label_encoded']
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=42,
    stratify=y_temp
)

# Tokenize with BERT
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)


class BERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




train_dataset = BERTDataset(train_encodings, y_train)
val_dataset = BERTDataset(val_encodings, y_val)
test_dataset = BERTDataset(test_encodings, y_test)

# DataLoaders (smaller batch size for memory!)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


class BioMedClassifier(nn.Module):
    def __init__(self, num_classes=13):
        super().__init__()
        self.bert = AutoModel.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_classes)
        self.criterion = nn.CrossEntropyLoss()  # Add loss function
    
    def forward(self, input_ids, attention_mask, labels=None):  # ← Add labels parameter!
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output
        x = self.dropout(pooled)
        logits = self.classifier(x)
        
        # Calculate loss if labels provided
        loss = None
        if labels is not None:
            loss = self.criterion(logits, labels)
        
        # Return in same format as AutoModel
        from collections import namedtuple
        Output = namedtuple('Output', ['loss', 'logits'])
        return Output(loss=loss, logits=logits)

model = BioMedClassifier(num_classes=13)

Original dataset: 16407 samples
Sampled dataset: 3281 samples (20% of each class)
⚡ Expected speedup: ~5x faster training!



/var/folders/cd/2c6x3jgj47j_fwl7231ts7m00000gn/T/ipykernel_40106/2824936658.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('specialty', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)).reset_index(drop=True)


## 🚀 Fine-tune

### TODO 3: Training loop

**Expected:** Fine-tune for 3-5 epochs

In [ ]:
# TODO 3: Fine-tune
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

n_epochs = 3  # Reduced from 5 for faster experimentation
best_val_f1 = 0.0

print(f"🚀 Starting training...")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Batches per epoch: {len(train_loader)}")
print(f"Expected time per epoch (CPU): ~20-30 minutes\n")

for epoch in range(n_epochs):
    print(f"{'='*60}")
    print(f"🔄 Epoch {epoch+1}/{n_epochs}")
    print(f"{'='*60}")
    
    # TRAINING
    model.train()
    train_loss = []
    
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # Model calculates loss automatically!
        
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        
        # Print progress every 100 batches
        if (batch_idx + 1) % 100 == 0:
            avg_loss = sum(train_loss[-100:]) / len(train_loss[-100:])
            print(f"  Batch [{batch_idx+1}/{len(train_loader)}] - Loss: {avg_loss:.4f}")
    
    print(f"✅ Training complete for epoch {epoch+1}")
    
    # VALIDATION
    model.eval()
    val_loss = []
    val_preds = []
    val_labels = []
    
    print(f"📊 Running validation...")
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            preds = torch.argmax(logits, dim=1)
            
            val_loss.append(loss.item())
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    val_f1 = f1_score(val_labels, val_preds, average='macro')
    
    print(f"\n📈 Epoch {epoch+1} Results:")
    print(f"  Train Loss: {sum(train_loss)/len(train_loss):.4f}")
    print(f"  Val Loss:   {sum(val_loss)/len(val_loss):.4f}")
    print(f"  Val F1:     {val_f1:.4f}")
    
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), '../models/biobert_best.pth')
        print(f"  ✅ New best model saved! (F1: {val_f1:.4f})")
    print()

🚀 Starting training...
Training samples: 1968
Validation samples: 656
Batches per epoch: 123
Expected time per epoch (CPU): ~20-30 minutes

🔄 Epoch 1/3


## 🤔 Reflection
1. Training time? GPU needed?
2. Beat baseline?

**Your reflection:**

*Write here*

## 📌 Summary
✅ Transformer fine-tuned  
✅ Performance evaluated

**Next:** `06_eval_and_error_analysis.ipynb`